In [1]:
import torch
import torchvision
import math

In [2]:
# Load observations from the mnist dataset. The observations are divided into a training set and a test set
mnist_train = torchvision.datasets.MNIST('./data', train=True, download=True)
x_train = mnist_train.data.reshape(-1, 1, 28, 28).float()  # torch.functional.nn.conv2d argument must include channels (1)
y_train = torch.zeros((mnist_train.targets.shape[0], 10))  # Create output tensor
y_train[torch.arange(mnist_train.targets.shape[0]), mnist_train.targets] = 1  # Populate output

mnist_test = torchvision.datasets.MNIST('./data', train=False, download=True)
x_test = mnist_test.data.reshape(-1, 1, 28, 28).float()  # torch.functional.nn.conv2d argument must include channels (1)
y_test = torch.zeros((mnist_test.targets.shape[0], 10))  # Create output tensor
y_test[torch.arange(mnist_test.targets.shape[0]), mnist_test.targets] = 1  # Populate output

/home/diderikk/.local/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
# Normalization of inputs
mean = x_train.mean()
std = x_train.std()
# Standard score = x - mean / std
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Divide training data into batches to speed up optimization
batches = 600
x_train_batches = torch.split(x_train, batches)
y_train_batches = torch.split(y_train, batches)

In [4]:
class ConvolutionalNeuralNetworkModel:
	def __init__(self):
		std = 1 / math.sqrt(1 * 5 * 5)
		# std = math.sqrt(6/(1 + 32)) # Xavier initializtion
		# 1 is the channel, 1 for greyscale and 3 for RGB 
		self.W1 = (torch.randn(32, 1, 5, 5) * std).requires_grad_()  # Shape: (out_channels, in_channels, image height, image width)
		self.b1 = (torch.randn(32) * std).requires_grad_()

		std = 1 / math.sqrt(32 * 5 * 5)
		# std = math.sqrt(6/(32 + 64)) # Xavier initializtion
		self.W2 = (torch.randn(64, 32, 5, 5) * std).requires_grad_()
		self.b2 = (torch.randn(64) * std).requires_grad_()

		std = 1 / math.sqrt(64 * 7 * 7)
		# std = math.sqrt(6/(64*7*7 + 10)) # Xavier initialization
		self.W3 = (torch.randn(64 * 7 * 7, 1024) * std).requires_grad_()  # Shape: (in_channels * image_pixels_after_max_pool, classes)
		self.b3 = (torch.randn(1024) * std).requires_grad_()

		std = 1 / math.sqrt(1024)
		self.W4 = (torch.randn(1024, 10) * std).requires_grad_()
		self.b4 = (torch.randn(10) * std).requires_grad_()

	def parameters(self):
		return [self.W1, self.b1, self.W2, self.b2, self.W3, self.b3, self.W4, self.b4]

	def logits(self, x):
		x = torch.nn.functional.conv2d(x, self.W1, self.b1, padding=2)
		x = torch.nn.functional.max_pool2d(x, kernel_size=2)
		x = torch.nn.functional.conv2d(x, self.W2, self.b2, padding=2)
		x = torch.nn.functional.max_pool2d(x, kernel_size=2)
		x = x.reshape(-1, 64 * 7 * 7) @ self.W3 + self.b3
		return x @ self.W4 + self.b4

	# Predictor
	def f(self, x):
		return torch.softmax(self.logits(x), dim=1)

	# Cross Entropy loss
	def loss(self, x, y):
		return torch.nn.functional.cross_entropy(self.logits(x), y.argmax(1))

	# Accuracy
	def accuracy(self, x, y):
		return torch.mean(torch.eq(self.f(x).argmax(1), y.argmax(1)).float())

In [5]:
model = ConvolutionalNeuralNetworkModel()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
for epoch in range(5):
    for batch in range(len(x_train_batches)):
        model.loss(x_train_batches[batch], y_train_batches[batch]).backward()  # Compute loss gradients
        optimizer.step()  # Perform optimization by adjusting W and b,
        optimizer.zero_grad()  # Clear gradients for next step

    print("accuracy = %s" % model.accuracy(x_test, y_test).item())

# 0.9831

/home/diderikk/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


accuracy = 0.9700000286102295
accuracy = 0.9790999889373779
accuracy = 0.9817000031471252
accuracy = 0.9824000000953674
accuracy = 0.98089998960495
